# Running Scripts in Juypter Lab

Python environment using Anaconda and Jupyter Notebook.

In [1]:
def sayHello(drone):
    print('Hello '+ drone)
sayHello("Tello")

Hello Tello


## Tello SDK 

The Tello SDK documentation can be found at the following link:

https://dl-cdn.ryzerobotics.com/downloads/tello/0228/Tello+SDK+Readme.pdf

### Download Packet Sender

[Packet Sender](https://packetsender.com/) is a free utility to for sending / receiving of network packets. Support for TCP, UDP, and SSL.


Default Gateway . . . . . . . . . : 192.168.10.1

[socket](https://docs.python.org/3/library/socket.html#module-socket) — Low-level networking interface

This module provides access to the BSD socket interface. It is available on all modern Unix systems, Windows, MacOS, and probably additional platforms.

In [2]:
# This example script demonstrates how to send a basic command over UDP to Tello
# We will use PacketSender to simulate Tello for the purposes of this demonstration.
# PacketSender makes it easy to test code that can then be run when connected to Tello.
# https://packetsender.com/

# This script is part of our course on Tello drone programming
# https://learn.droneblocks.io/p/tello-drone-programming-with-python/

# Import the built-in socket package
import socket

# IP and port of sending computer
# In this case we're sending a UDP packet to PacketSender for demonstration purposes
# Be sure to change this to the IP address of the computer running this Python script

# IP address of the local computer 
# - Mac Terminal command ifconfip
# - Windows Command Prompt ipconfig
simulated_tello_address = ('192.168.10.2', 8889)

# Create a UDP connection that we'll send the command to
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Command variable that we'll send
# This "command" value is what lets Tello know that we want to enter command mode
message = "command"

# Send the message to Tello
sock.sendto(message.encode(), simulated_tello_address)

# Print message to screen
print("Message sent!")

Message sent!


## Sending and Receiving UDP Packets

[time](https://docs.python.org/3/library/time.html#module-time) — Time access and conversions

This module provides various time-related functions. For related functionality, see also the datetime and calendar modules.

In [3]:
# This example script demonstrates how to send/receive commands to/from Tello
# This script is part of our course on Tello drone programming
# https://learn.droneblocks.io/p/tello-drone-programming-with-python/

# Import the built-in socket and time modules
import socket
import time

# IP and port of Tello
tello_address = ('192.168.10.1', 8889)

# Create a UDP connection that we'll send the command to
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Let's be explicit and bind to a local port on our machine where Tello can send messages
sock.bind(('', 9000))

# Function to send messages to Tello
def send(message):
  try:
    sock.sendto(message.encode(), tello_address)
    print("Sending message: " + message)
  except Exception as e:
    print("Error sending: " + str(e))

# Function that listens for messages from Tello and prints them to the screen
def receive():
  try:
    response, ip_address = sock.recvfrom(128)
    print("Received message: " + response.decode(encoding='utf-8') + " from Tello with IP: " + str(ip_address))
  except Exception as e:
    print("Error receiving: " + str(e))


# Send Tello into command mode
send("command")

# Receive response from Tello
receive()

# Delay 3 seconds before we send the next command
time.sleep(3)

# Ask Tello about battery status
send("battery?")

# Receive battery response from Tello
receive()

# Close the UDP socket
sock.close()

Sending message: command
Received message: ok from Tello with IP: ('192.168.10.1', 8889)
Sending message: battery?
Received message: 39
 from Tello with IP: ('192.168.10.1', 8889)


## Tello Missions

An Intro to Threading in Python - [Learn more](https://realpython.com/intro-to-python-threading/)

Python threading allows you to have different parts of your program run concurrently and can simplify your design.

**daemon threads**

Daemons are only useful when the main program is running, and it's okay to kill them off once the other non-daemon threads have exited. https://pycozmo.readthedocs.io/en/stable/generated/pycozmo.conn.html#pycozmo.conn.Connection.daemon

In [4]:
# Import the necessary modules
import socket
import threading
import time

# IP and port of Tello
tello_address = ('192.168.10.1', 8889)

# IP and port of local computer
local_address = ('', 9000)

# Create a UDP connection that we'll send the command to
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Bind to the local address and port
sock.bind(local_address)

# Send the message to Tello and allow for a delay in seconds
def send(message, delay):
  # Try to send the message otherwise print the exception
  try:
    sock.sendto(message.encode(), tello_address)
    print("Sending message: " + message)
  except Exception as e:
    print("Error sending: " + str(e))

  # Delay for a user-defined period of time
  time.sleep(delay)

# Receive the message from Tello
def receive():
  # Continuously loop and listen for incoming messages
  while True:
    # Try to receive the message otherwise print the exception
    try:
      response, ip_address = sock.recvfrom(128)
      print("Received message: " + response.decode(encoding='utf-8'))
    except Exception as e:
      # If there's an error close the socket and break out of the loop
      sock.close()
      print("Error receiving: " + str(e))
      break

# Create and start a listening thread that runs in the background
# This utilizes our receive functions and will continuously monitor for incoming messages
receiveThread = threading.Thread(target=receive)
receiveThread.daemon = True
receiveThread.start()

#Door mission
doorlength = 127
doorwidth = 91.44
send("command", 3)
send("takeoff", 5)
send("up" + str(doorlength / 2), 4)
send("right" + str(doorwidth), 4)
send("down" + str(doorlength), 4)
send("left" + str(doorwidth), 4)
send("up" + str(doorlength / 2), 4)
send("land", 5)
sock.close()

Sending message: command
Received message: ok
Sending message: takeoff
Sending message: up63.5
Received message: unknown command: up63.5
Received message: ok
Sending message: right91.44
Received message: unknown command: right91.44
Sending message: down127
Received message: unknown command: down127
Sending message: left91.44
Received message: unknown command: left91.44
Sending message: up63.5
Received message: unknown command: up63.5
Sending message: land
Received message: ok
Error receiving: [WinError 10038] An operation was attempted on something that is not a socket
